In [1]:
# import algemeen
from pathlib import Path
import math
import pandas as pd

# imports base modules
from toolbox_continu_inzicht import Config, DataAdapter


In [2]:
# lees configuratiebestand
yaml_config_file = "continu-inzicht-demo-zorgplicht-create-db.yaml"
data_path = Path.joinpath(Path.cwd(), "data", yaml_config_file)

config = Config(config_path=data_path)
config.lees_config()

In [3]:
# data adapter aanmaken
data_adapter = DataAdapter(config=config)

**Reset database**

In [4]:
from toolbox_continu_inzicht.helpers import reset_database

tables = [
    {"name": "conditions", "reset_index": True},
    {"name": "expertjudgements", "reset_index": True},
    {"name": "failuremechanism", "reset_index": True},
    {"name": "fragilitycurves", "reset_index": True},
    {"name": "fragilitycurves_data", "reset_index": False},
    {"name": "fragilitycurves_intergrate", "reset_index": True},
    {"name": "fragilitycurves_intergrate_data", "reset_index": False},
    {"name": "hydraulicload_locations", "reset_index": True},
    {"name": "sections", "reset_index": True},
    {"name": "statistics", "reset_index": True},
    {"name": "statistics_data", "reset_index": False},
    {"name": "variants", "reset_index": True},
]

try:
    reset_database(data_adapter=data_adapter, tables=tables, output="reset_database")

except Exception as e:
    print(f"{e}")

**Faalmechanismes toevoegen**

In [5]:
df_failuremechanism = data_adapter.input(input="in_failuremechanism_csv")

# sla de faalmechanismes op in de CI database (tabel "failuremechanism") via de data_adapter "out_failuremechanism_table"
data_adapter.output(output="out_failuremechanism_table", df=df_failuremechanism)

**Condities toevoegen**

In [6]:
df_conditions = data_adapter.input(input="in_conditions_csv")

# sla de condities op in de CI database (tabel "conditions") via de data_adapter "out_conditions_table"
data_adapter.output(output="out_conditions_table", df=df_conditions)

**Dijkvakken toevoegen**

In [7]:
# lees de secties uit een geojson-bestand via de data_adapter "in_sections_geojson"
df_secties = data_adapter.input(input="in_sections_geojson")

# Hernoem de kolom "oude_naam" naar "nieuwe_naam"
df_secties = df_secties.rename(
    columns={
        "stationid": "hydraulicload_location_id",
        "belasting": "overleefde_belasting",
    }
)

# sla de secties op in de CI database (tabel "sections") via de data_adapter "out_sections_table"
data_adapter.output(output="out_sections_table", df=df_secties)

**Hydraulic load locaties toevoegen**

In [8]:
df_measuringstations = data_adapter.input(input="in_hydraulicload_locations_geojson")


# sla de meetstations op in de CI database (tabel "hydraulicload_locations") via de data_adapter "out_measuringstations_table"
data_adapter.output(output="out_hydraulicload_locations_table", df=df_measuringstations)

**Statistics toevoegen**

In [9]:
df_statistics = data_adapter.input(input="in_statistics_csv")

# sla de statistics op in de CI database (tabel "statistics" en "statistics_data") via de data_adapter "out_statistics_table"
data_adapter.output(output="out_statistics_table", df=df_statistics)

**Varianten toevoegen**

In [10]:
df_variants = data_adapter.input(input="in_variants_csv")

# sla de varianten op in de CI database (tabel "variants") via de data_adapter "out_variants_table"
data_adapter.output(output="out_variants_table", df=df_variants)

**Fragility curves toevoegen**

In [11]:
df_fragilitycurves = data_adapter.input(input="in_fragilitycurves_csv")

# sla de statistics op in de CI database (tabel "fragilitycurves" en "fragilitycurves_data") via de data_adapter "out_fragilitycurves_table"
data_adapter.output(output="out_fragilitycurves_table", df=df_fragilitycurves)

**Beheerdersoordelen toevoegen**

In [12]:
df_expertjudgements = data_adapter.input(input="in_expertjudgements_csv")

# sla de beheerdersoordelen op in de CI database (tabel "expertjudgements") via de data_adapter "out_expertjudgements_table"
data_adapter.output(output="out_expertjudgements_table", df=df_expertjudgements)

**Fragilitycurves intergrate toevoegen**

Controleer of er voor de afgeleide beheerdersoordelen de fragility curves 
Overgenomen zijn van de basis beheerdersoordeel.

Als er een 'Load_limit' en 'probability_contribution_reductionfactor' zijn opgegeven worden deze gebruikt om de curve aan te passen


In [13]:
from toolbox_continu_inzicht.base.fragility_curve import FragilityCurve

# ophalen alle door te rekenen berekeningen.
df_calculate_list = data_adapter.input("in_calculate_list")

# Filter toepassen op de DataFrame (bijv. leeftijd groter dan 24)
df_filtered = df_calculate_list[~df_calculate_list["has_fragilitycurves"]]

for index, calculate_item in df_filtered.iterrows():
    try:
        section_id = int(calculate_item["section_id"])
        expertjudgement_id = int(calculate_item["expertjudgement_id"])
        has_fragilitycurves = bool(calculate_item["has_fragilitycurves"])
        has_fragilitycurves_base = bool(calculate_item["has_fragilitycurves_base"])

        fragilitycurves_id = int(calculate_item["fragilitycurves_id"])
        fragilitycurves_base_id = int(calculate_item["fragilitycurves_base_id"])

        failuremechanism_id = calculate_item["failuremechanism_id"]
        statistics_id = int(calculate_item["statistic_id"])
        hydraulicload_location_id = int(calculate_item["hydraulicload_location_id"])

        has_intergrate = bool(calculate_item["has_intergrate"])
        intergrate_id = calculate_item["intergrate_id"]
        load_limit = calculate_item["load_limit"]
        probability_contribution_reductionfactor = calculate_item[
            "probability_contribution_reductionfactor"
        ]

        if math.isnan(intergrate_id):
            intergrate_id = int(0)
        else:
            intergrate_id = int(intergrate_id)

        if math.isnan(load_limit):
            load_limit = float(0)
        else:
            load_limit = float(load_limit)

        if math.isnan(probability_contribution_reductionfactor):
            probability_contribution_reductionfactor = float(1)
        else:
            probability_contribution_reductionfactor = float(
                probability_contribution_reductionfactor
            )

        # TODO?
        if math.isnan(failuremechanism_id):
            failuremechanism_id = int(1)
        else:
            failuremechanism_id = int(failuremechanism_id)

        # bepaal curve parameters
        config.data_adapters["in_fragilitycurves_data"]["section_id"] = section_id
        config.data_adapters["in_fragilitycurves_data"]["failuremechanism_id"] = (
            failuremechanism_id
        )
        config.data_adapters["in_fragilitycurves_data"]["fragilitycurves_id"] = (
            fragilitycurves_base_id
        )

        # laad basis curve
        fragility_curve = FragilityCurve(data_adapter=data_adapter)
        fragility_curve.load("in_fragilitycurves_data")
        df_fragility_curve = fragility_curve.as_dataframe()

        # moet er nog een curve afgeleid worden
        if has_fragilitycurves_base:
            if has_intergrate:
                # eventueel fragilitycurve aanpassen
                load_limit_curve = df_fragility_curve["hydraulicload"].min()
                if load_limit > load_limit_curve:
                    fragility_curve.reliability_update(
                        update_level=load_limit,
                        trust_factor=probability_contribution_reductionfactor,
                    )

            df_fragility_curve["id"] = fragilitycurves_id
            df_fragility_curve["name"] = f"fragilitycurve_{fragilitycurves_id}"
            df_fragility_curve["section_id"] = section_id
            df_fragility_curve["failuremechanism_id"] = failuremechanism_id

            data_adapter.output(output="out_fragilitycurves_new", df=df_fragility_curve)

    except Exception as e:
        print(f"FL: {calculate_item}: {e}")

**Toevoegen ontbrekende integrate**

Als deze initieel niet zijn toegevoegd kunnen deze toegevoegd worden met 'probability_contribution_reductionfactor' op 1.0 en de 'load_limit' op laagste waterstand

In [14]:
# ophalen alle door te rekenen berekeningen.
df_calculate_list = data_adapter.input("in_calculate_list")

# Filter toepassen op de DataFrame (bijv. leeftijd groter dan 24)
df_filtered = df_calculate_list[~df_calculate_list["has_intergrate"]]

for index, calculate_item in df_filtered.iterrows():
    try:
        section_id = int(calculate_item["section_id"])
        expertjudgement_id = int(calculate_item["expertjudgement_id"])
        fragilitycurves_id = int(calculate_item["fragilitycurves_id"])
        fragilitycurves_base_id = int(calculate_item["fragilitycurves_base_id"])
        failuremechanism_id = calculate_item["failuremechanism_id"]

        # bepaal curve parameters
        config.data_adapters["in_fragilitycurves_data"]["section_id"] = section_id
        config.data_adapters["in_fragilitycurves_data"]["failuremechanism_id"] = (
            failuremechanism_id
        )
        config.data_adapters["in_fragilitycurves_data"]["fragilitycurves_id"] = (
            fragilitycurves_base_id
        )

        # laad basis curve
        fragility_curve = FragilityCurve(data_adapter=data_adapter)
        fragility_curve.load("in_fragilitycurves_data")
        df_fragility_curve = fragility_curve.as_dataframe()
        # intergrate toevoegen expertjudgement_id, section_id, failuremechanism_id, load_limit, probability_contribution_reductionfactor

        load_limit = df_fragility_curve["hydraulicload"].min()
        probability_contribution_reductionfactor = 1.0

        data = {
            "expertjudgement_id": [expertjudgement_id],
            "section_id": [section_id],
            "failuremechanism_id": [failuremechanism_id],
            "load_limit": [load_limit],
            "probability_contribution_reductionfactor": [
                probability_contribution_reductionfactor
            ],
        }

        # DataFrame maken
        df_intergrate = pd.DataFrame(data)

        data_adapter.output(
            output="out_fragilitycurves_intergrate_new", df=df_intergrate
        )

    except Exception as e:
        print(f"FL: {calculate_item}: {e}")

**Voeg de data van de Integrate toe in de database**

Voor de ontbrekende intergrate (data) wordt met de fragility curve en de statistieken wordt de data van de intergrate toegevoegd in de database.

In [15]:
from toolbox_continu_inzicht.fragility_curves import IntegrateFragilityCurve

integrate_fragility_curve = IntegrateFragilityCurve(data_adapter=data_adapter)

# ophalen alle door te rekenen berekeningen.
df_calculate_list = data_adapter.input("in_calculate_list")

# Filter toepassen op de DataFrame
df_filtered = df_calculate_list[~df_calculate_list["has_intergrate_data"]]

for index, calculate_item in df_filtered.iterrows():
    try:
        section_id = int(calculate_item["section_id"])
        expertjudgement_id = int(calculate_item["expertjudgement_id"])
        has_fragilitycurves = bool(calculate_item["has_fragilitycurves"])
        has_fragilitycurves_base = bool(calculate_item["has_fragilitycurves_base"])

        fragilitycurves_id = int(calculate_item["fragilitycurves_id"])
        fragilitycurves_base_id = int(calculate_item["fragilitycurves_base_id"])

        failuremechanism_id = calculate_item["failuremechanism_id"]
        statistics_id = int(calculate_item["statistic_id"])
        hydraulicload_location_id = int(calculate_item["hydraulicload_location_id"])

        has_intergrate = bool(calculate_item["has_intergrate"])
        intergrate_id = calculate_item["intergrate_id"]
        load_limit = calculate_item["load_limit"]

        # bepaal curve parameters
        config.data_adapters["in_fragilitycurves_data"]["section_id"] = section_id
        config.data_adapters["in_fragilitycurves_data"]["failuremechanism_id"] = (
            failuremechanism_id
        )
        config.data_adapters["in_fragilitycurves_data"]["fragilitycurves_id"] = (
            fragilitycurves_base_id
        )

        # bepaal statistiek parameters
        config.data_adapters["in_statistics_data"]["statistics_id"] = statistics_id
        config.data_adapters["in_statistics_data"]["hydraulicload_location_id"] = (
            hydraulicload_location_id
        )

        # maak de intergrate
        integrate_fragility_curve.run(
            input=["in_statistics_data", "in_fragilitycurves_data"],
            output="df_integrate_fragility_curve",
        )

        df_integrate_fragility_curve = integrate_fragility_curve.df_out
        df_integrate_fragility_curve["intergrate_id"] = intergrate_id

        data_adapter.output(
            output="out_fragilitycurves_intergrate_data",
            df=df_integrate_fragility_curve,
        )

    except Exception as e:
        print(f"FL: {calculate_item}: {e}")